In [1]:
import numpy as np

import time
import mujoco
import mujoco.viewer

import mujoco
import numpy as np
import matplotlib.pyplot as plt
import mujoco.viewer

import mediapy as media
import time

import matplotlib.pyplot as plt
from scipy.linalg import null_space
from scipy.spatial.transform import Rotation as R

In [2]:
from main_22aug import LeapNodeMujoco, GraspClass
from inv_kin import GradientDescentIK

model_path = '/home/sysidea/leap_hand_mujoco/model/leap hand/leaphand_19.8.24.xml'
leap_hand = LeapNodeMujoco(model_path)

IK_index=GradientDescentIK('/home/sysidea/leap_hand_mujoco/model/leap hand/index_finger.xml')
IK_thumb=GradientDescentIK('/home/sysidea/leap_hand_mujoco/model/leap hand/thumb.xml')

In [3]:
camera = mujoco.MjvCamera()
mujoco.mjv_defaultFreeCamera(leap_hand.m, camera)

# Set camera position
camera.lookat[:] = np.array([0.0, 0.0, 0.0])  # Point in space the camera is looking at
camera.azimuth = -90                # Horizontal angle in degrees
camera.elevation = -10                      # Vertical angle in degrees
camera.distance = 1                         # Distance from the lookat point


In [4]:
mujoco.mj_step(leap_hand.m,leap_hand.d)

# site_namei1 = 'contact_index1'
# site_namet1 = 'contact_thumb1'
# site_namei2 = 'contact_index2'
# site_namet2 = 'contact_thumb2'

# site_idi1= IK_index.model.site(site_namei1).id
# site_idt1= IK_thumb.model.site(site_namet1).id
# site_idi2= IK_index.model.site(site_namei2).id
# site_idt2= IK_thumb.model.site(site_namet2).id

pos_worldi1=leap_hand.d.site_xpos[0].reshape(3)
rot_worldi1=leap_hand.d.site_xmat[0].reshape(3,3)
pos_worldt1=leap_hand.d.site_xpos[1].reshape(3)
rot_worldt1=leap_hand.d.site_xmat[1].reshape(3,3)

pos_worldi2=leap_hand.d.site_xpos[2].reshape(3)
rot_worldi2=leap_hand.d.site_xmat[2].reshape(3,3)
pos_worldt2=leap_hand.d.site_xpos[3].reshape(3)
rot_worldt2=leap_hand.d.site_xmat[3].reshape(3,3)



# print(leap_hand.d.site(site_id).xpos.reshape(3))
# print(leap_hand.d.site(site_id).xmat.reshape(3,3))


pos_palmi1=pos_worldi1-leap_hand.d.body(leap_hand.m.body('palm').id).xpos.reshape(3)
rot_palmi1=np.reshape(leap_hand.d.body(leap_hand.m.body('palm').id).xmat,[3,3]).T @ rot_worldi1
pos_palmt1=pos_worldt1-leap_hand.d.body(leap_hand.m.body('palm').id).xpos.reshape(3)
rot_palmt1=np.reshape(leap_hand.d.body(leap_hand.m.body('palm').id).xmat,[3,3]).T @ rot_worldt1

pos_palmi2=pos_worldi2-leap_hand.d.body(leap_hand.m.body('palm').id).xpos.reshape(3)
rot_palmi2=np.reshape(leap_hand.d.body(leap_hand.m.body('palm').id).xmat,[3,3]).T @ rot_worldi2
pos_palmt2=pos_worldt1-leap_hand.d.body(leap_hand.m.body('palm').id).xpos.reshape(3)
rot_palmt2=np.reshape(leap_hand.d.body(leap_hand.m.body('palm').id).xmat,[3,3]).T @ rot_worldt2
# goal_pos_index = [0.0565407, 0.0226578, -0.07722874]  # desired position of the end effector
# goal_rot_index=np.array([ [1.00000000e+00, -3.92523115e-16,  0.00000000e+00],  [0.00000000e+00,
#   0.00000000e+00, -1.00000000e+00],  [3.92523115e-16,  1.00000000e+00,
#   0.00000000e+00]])

In [5]:
result_index1=IK_index.calculate(pos_palmi1, rot_palmi1,'contact_index')
result_thumb1=IK_thumb.calculate(pos_palmt1, rot_palmt1,'contact_thumb')
result_index2=IK_index.calculate(pos_palmi2, rot_palmi2,'contact_index')
result_thumb2=IK_thumb.calculate(pos_palmt2, rot_palmt2,'contact_thumb')


framerate=30

#go near the contact point
while leap_hand.d.time < 2:

    leap_hand.apply_controls_palm(0)
    leap_hand.apply_controls_wrist(0)

    #placeholder code 
    #index finger
    [a,b,c,d]=result_index1
    [e,f,g,h]=result_thumb1
    
    leap_hand.apply_controls_hand([a,b,c,d,0,0,0,0,0,0,0,0,e,f,g,h])
    
    #temperory
    leap_hand.step_video(framerate,camera)

while leap_hand.d.time < 4 and leap_hand.d.time>2:

    leap_hand.apply_controls_palm(0)
    leap_hand.apply_controls_wrist(0)

    #placeholder code 
    #index finger
    [a,b,c,d]=result_index2
    [e,f,g,h]=result_thumb2
    
    leap_hand.apply_controls_hand([a,b,c,d,0,0,0,0,0,0,0,0,e,f,g,h])
    
    #temperory
    leap_hand.step_video(framerate,camera)

while leap_hand.d.time < 6 and leap_hand.d.time>4:

    leap_hand.apply_controls_palm(0.2)
    leap_hand.apply_controls_wrist(0)

    #placeholder code 
    #index finger
    [a,b,c,d]=result_index2
    [e,f,g,h]=result_thumb2
    
    leap_hand.apply_controls_hand([a,b,c,d,0,0,0,0,0,0,0,0,e,f,g,h])
    
    #temperory
    leap_hand.step_video(framerate,camera)

while leap_hand.d.time < 8 and leap_hand.d.time>6:

    leap_hand.apply_controls_palm(0.2)
    leap_hand.apply_controls_wrist(0)

    #placeholder code 
    #index finger
    [a,b,c,d]=result_index2
    [e,f,g,h]=result_thumb2
    
    leap_hand.apply_controls_hand([a,b,c,d,0,0,0,0,0,0,0,0,e,f,g,h])
    
    #temperory
    leap_hand.step_video(framerate,camera)

    

leap_hand.play_video(framerate)

Total frames captured: 241


This browser does not support the video tag.


In [6]:
# framerate=30

# #go near the contact point
# while leap_hand.d.time < 2:

#     leap_hand.apply_controls_palm(0)
#     leap_hand.apply_controls_wrist(0)

#     #placeholder code 
#     #index finger
#     # [a,b,c,d]=LHK.perform_ik('fingertip',kdl.Frame(kdl.Rotation.RPY(0, 0, 0),
#     #                         kdl.Vector(0, 0, 0)),)
#     # #thumb
#     # [e,f,g,h]=LHK.perform_ik('thumb_fingertip',kdl.Frame(kdl.Rotation.RPY(0, 0, 0),
#     #                         kdl.Vector(0, 0, 0)),)
    
#     # leap_hand.apply_controls([a,b,c,d,0,0,0,0,0,0,0,0,e,f,g,h])
    
#     #temperory
#     leap_hand.apply_controls([0.88168, 0, 0.438445, 0.236, 0, 0, 0, 0, 0, 0, 0, 0, 1.54432, 0, -0.7815, 0.7852])

#     leap_hand.step_video(framerate,'closeup')
       
# #apply force to hold object
# while leap_hand.d.time < 4 and leap_hand.d.time>2:
    
#     leap_hand.apply_controls_palm(0)
#     leap_hand.apply_controls_wrist(0)
    
#     #placeholder code
#     #leap_hand.d.ctrl[2:6]=null_space(projection_matrix)

#     leap_hand.step_video(framerate,'closeup')

# #lift object
# while leap_hand.d.time > 4 and leap_hand.d.time<6:
    
#     leap_hand.apply_controls_palm(0.2)
#     leap_hand.apply_controls_wrist(0)

#     #placeholder
#     #leap_hand.d.ctrl[2:6]=null_space(projection_matrix)
    
#     leap_hand.step_video(framerate,'closeup')

# #rotate palm
# while leap_hand.d.time > 6 and leap_hand.d.time<8:

#     leap_hand.apply_controls_wrist(3.14)
    
#     #placeholder
#     #leap_hand.d.ctrl[2:6]=null_space(projection_matrix)
    
#     leap_hand.step_video(framerate,'closeup')

# #bring object down to palm
# while leap_hand.d.time > 8 and leap_hand.d.time<10:
    
#     leap_hand.apply_controls_palm(1)
#     leap_hand.apply_controls_wrist(3.14)

#     #placeholder
#     #leap_hand.d.ctrl[2:6]=null_space(projection_matrix)
    
#     leap_hand.step_video(framerate,'closeup')

# # #drop object
# # while leap_hand.d.time > 8 and leap_hand.d.time<10:
    
# #     leap_hand.apply_controls_palm(1)
# #     leap_hand.apply_controls_wrist(3.14)

# #     #leap_hand.apply_controls(np.zeros(16))
    
# #     leap_hand.step_video(framerate)

# # #use thumb to push object
# # while leap_hand.d.time > 8 and leap_hand.d.time<10:
    
# #     leap_hand.apply_controls_palm(1)
# #     leap_hand.apply_controls_wrist(3.14)

# #      #leap_hand.torque_control(null_space(I-pinv(G)*G)+pinv(G)*(Kp*(X-Xd)))
    
# #     leap_hand.step_video(framerate)

# # #middle and last finger flat end on cube
# # while leap_hand.d.time > 10 and leap_hand.d.time<12:
    
# #     leap_hand.apply_controls_palm(1)
# #     leap_hand.apply_controls_wrist(3.14)

# #      #leap_hand.inv_kin(mid,other)
    
# #     leap_hand.step_video(framerate)

# # #apply force
# # while leap_hand.d.time > 10 and leap_hand.d.time<12:
    
# #     leap_hand.apply_controls_palm(1)
# #     leap_hand.apply_controls_wrist(3.14)

# #      #Tau=J.T*F
    
# #     leap_hand.step_video(framerate)
        
    
# leap_hand.play_video(framerate)
    

Grasp Matrix and Jacobian (Index and Thumb)

In [7]:
# # Define matrices
# n=2
# rpalm_inv=np.linalg.inv(np.reshape(leap_hand.d.xmat[3],(3,3)))
# r_cube=np.reshape(leap_hand.d.xmat[0],(3,3))
# r_theta=rpalm_inv*r_cube
# contact_orientations=[np.reshape(leap_hand.d.site_xmat[0],(3,3)),np.reshape(leap_hand.d.site_xmat[0],(3,3))]
# bs=[np.reshape([0, 0.027,0],(3,1)),np.reshape([0, -0.027,0],(3,1))]

# Grasp=GraspClass()
# G_index_thumb = Grasp.G(n,contact_orientations,r_theta,bs)
# # Jh_index_thumb = np.zeros([6, 3])
# # I = np.eye(G.shape[1])

# # # Compute the pseudoinverse of G
# # pinv_G = np.linalg.pinv(G)

# # # Compute the projection matrix (I - pinv_G @ G)
# # projection_matrix = I - pinv_G @ G

# # # # Compute the null space of the projection matrix
# # print(null_space(projection_matrix))
# print(G_index_thumb)
